In [ ]:
import requests
import time

wait_time = 5
num_retries = 3
for retry_count in range(num_retries):
    try:
        # response = requests.put('https://api.sampleapis.com/wines/whites')
        response = requests.put('https://api.sampleapis.com/93239-w', timeout=30)
        response.raise_for_status()
    except Exception as e:
        print(f'error occurred: {e}')
        if response.status_code == 429:
            if retry_count == num_retries-1:
                print(f'All retries exhausted. ending the request procedure')
            else:
                print(f'retrying after {wait_time} secs... (retry count={retry_count+1})')
                time.sleep(wait_time)
        else:
            if retry_count == num_retries-1:
                print(f'All retries exhausted. ending the request procedure')
            else:
                print(f'retrying ... (retry count={retry_count+1})')


print(response)

error occurred: 404 Client Error: Not Found for url: https://api.sampleapis.com/93239-w
retrying after 5 secs... (retry count=1)
error occurred: 404 Client Error: Not Found for url: https://api.sampleapis.com/93239-w
retrying after 5 secs... (retry count=2)
error occurred: 404 Client Error: Not Found for url: https://api.sampleapis.com/93239-w
All retries exhausted. ending the request procedure
<Response [404]>


In [45]:
response.text

'[{"winery":"Domaine de La Romanée-Conti","wine":"Montrachet Grand Cru 2010","rating":{"average":"4.9","reviews":"37 ratings"},"location":"France\\n·\\nMontrachet Grand Cru","image":"https://images.vivino.com/thumbs/rORmihtxSrKG7SfuI0bD6w_pb_x300.png","id":1},{"winery":"Domaine de La Romanée-Conti","wine":"Montrachet Grand Cru 2014","rating":{"average":"4.9","reviews":"33 ratings"},"location":"France\\n·\\nMontrachet Grand Cru","image":"https://images.vivino.com/thumbs/rORmihtxSrKG7SfuI0bD6w_pb_x300.png","id":2},{"winery":"Domaine Coche-Dury","wine":"Meursault Les Rougeots 2001","rating":{"average":"4.9","reviews":"25 ratings"},"location":"France\\n·\\nMeursault","image":"https://images.vivino.com/thumbs/l5W5NRvZR_SzClIDSnG5Ag_pb_x300.png","id":3},{"winery":"Domaine Coche-Dury","wine":"Corton-Charlemagne Grand Cru N.V.","rating":{"average":"4.8","reviews":"416 ratings"},"location":"France\\n·\\nCorton-Charlemagne Grand Cru","image":"https://images.vivino.com/thumbs/ZGxHdQyGQt-hfJt7eNMX

In [46]:
import json

In [47]:
resp_json = json.loads(response.text)

In [48]:
resp_json

[{'winery': 'Domaine de La Romanée-Conti',
  'wine': 'Montrachet Grand Cru 2010',
  'rating': {'average': '4.9', 'reviews': '37 ratings'},
  'location': 'France\n·\nMontrachet Grand Cru',
  'image': 'https://images.vivino.com/thumbs/rORmihtxSrKG7SfuI0bD6w_pb_x300.png',
  'id': 1},
 {'winery': 'Domaine de La Romanée-Conti',
  'wine': 'Montrachet Grand Cru 2014',
  'rating': {'average': '4.9', 'reviews': '33 ratings'},
  'location': 'France\n·\nMontrachet Grand Cru',
  'image': 'https://images.vivino.com/thumbs/rORmihtxSrKG7SfuI0bD6w_pb_x300.png',
  'id': 2},
 {'winery': 'Domaine Coche-Dury',
  'wine': 'Meursault Les Rougeots 2001',
  'rating': {'average': '4.9', 'reviews': '25 ratings'},
  'location': 'France\n·\nMeursault',
  'image': 'https://images.vivino.com/thumbs/l5W5NRvZR_SzClIDSnG5Ag_pb_x300.png',
  'id': 3},
 {'winery': 'Domaine Coche-Dury',
  'wine': 'Corton-Charlemagne Grand Cru N.V.',
  'rating': {'average': '4.8', 'reviews': '416 ratings'},
  'location': 'France\n·\nCorton-

In [58]:
import logging
import pandas as pd
def convert_json_to_csv(json_body, file_name='temp_wine.csv'):
    #check if input is of list type
    if type(json_body) is not list:
        logging.error('expected list of json type dictionaries. got %s',type(json_body))
        return None

    # create a empty dictionary
    df_dict = {'id': [], 'wine':[], 'rating':[], 'location':[], 'image':[] }

    # iterate over the json_body to create a dictionary of wine details
    for json_dict in json_body:
        df_dict['id'].append(json_dict.get('id', 0))
        df_dict['wine'].append(json_dict.get('wine'))
        df_dict['rating'].append(json_dict.get('rating'))
        if json_dict.get('location'):
            location = json_dict.get('location').replace('\n','')
            df_dict['location'].append(location)
        else:
            df_dict['location'].append(None)
        df_dict['image'].append(json_dict.get('image'))

    # create a data frame out of the dictionary from previous step
    wine_df = pd.DataFrame(data=df_dict)
    wine_df.reset_index(drop=True, inplace=True)
    # create a file from the dataframe
    wine_df.to_csv(file_name, index=False)
    return  wine_df
    logging.info('Successfully created file %s',file_name)


In [59]:
wine_df = convert_json_to_csv(resp_json)

In [60]:

wine_df

,id,wine,rating,location,image
0,1,Montrachet Grand Cru 2010,"{'average': '4.9', 'reviews': '37 ratings'}",France·Montrachet Grand Cru,https://images.vivino.com/thumbs/rORmihtxSrKG7...
1,2,Montrachet Grand Cru 2014,"{'average': '4.9', 'reviews': '33 ratings'}",France·Montrachet Grand Cru,https://images.vivino.com/thumbs/rORmihtxSrKG7...
2,3,Meursault Les Rougeots 2001,"{'average': '4.9', 'reviews': '25 ratings'}",France·Meursault,https://images.vivino.com/thumbs/l5W5NRvZR_SzC...
3,4,Corton-Charlemagne Grand Cru N.V.,"{'average': '4.8', 'reviews': '416 ratings'}",France·Corton-Charlemagne Grand Cru,https://images.vivino.com/thumbs/ZGxHdQyGQt-hf...
4,5,Estate Finch Hollow Chardonnay (Cave Fermented...,"{'average': '4.8', 'reviews': '113 ratings'}",United States·Napa Valley,https://images.vivino.com/thumbs/pQ_92smWRKG7Y...
...,...,...,...,...,...
499,500,Chevalier-Montrachet Grand Cru 2015,"{'average': '4.6', 'reviews': '27 ratings'}",France·Chevalier-Montrachet Grand Cru,https://images.vivino.com/thumbs/lD32nuv9Sw6sJ...
500,501,Puligny-Montrachet 1er Cru Champ Canet 2011,"{'average': '4.6', 'reviews': '27 ratings'}",France·Puligny-Montrachet Premier Cru,https://images.vivino.com/thumbs/cDjOBVU0QW-ur...
501,502,Auxey-Duresses 2004,"{'average': '4.6', 'reviews': '27 ratings'}",France·Auxey-Duresses,https://images.vivino.com/thumbs/OzI_mGYSQdCoy...
502,503,Yattarna Chardonnay (BIN 144) 2003,"{'average': '4.6', 'reviews': '27 ratings'}",Australia·Adelaide Hills,https://images.vivino.com/thumbs/aMdgLSKeQVybJ...


In [ ]:
source(file/API call) --> data cleaning/data validation --> load_table/stage_table --> intermediate_table (transformation) --> final_table




select * from load_table
where (id, timestamp) not in (select id, timestamp 
                              from intermediate_table)